Limpeza e Filtragem
O Problema: Temos gigabytes de dados misturados (Renda Fixa, Cambial, Multimercado). O desafio pede especificamente a "classe de Ações".

Script:

1 - Ler o arquivo de Cadastro (cad_fi.csv) para pegar os CNPJs que são de "Ações".
2 - Ler os arquivos mensais (inf_diario...) um por um.
3 - Filtrar apenas as linhas que têm esses CNPJs.
4 - Salvar tudo num arquivo único e otimizado (formato Parquet, que é muito mais rápido que CSV para leitura posterior).

Importamos as bibliotecas e definimos os caminhos. Usaremos o cad_fi.csv para filtrar os fundos de interesse antes de processar as séries temporais pesadas.

## 1. Imports e Configuração
Importamos as bibliotecas e definimos os caminhos. Usaremos o cad_fi.csv para filtrar os fundos de interesse antes de processar as séries temporais pesadas.

In [8]:
import pandas as pd
import glob
import os
import gc # Garbage Collector para limpar memória RAM

# Caminhos
RAW_DIR = '../data/raw'
PROCESSED_DIR = '../data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)

## 2. Filtrando os CNPJs de Ações

Segundo o escopo do desafio, devemos utilizar a classe de Ações. Carregamos o cadastro (cad_fi.csv), filtramos por CLASSE == 'Fundo de Ações' e status EM FUNCIONAMENTO NORMAL. Isso gera uma lista de CNPJs válidos ("White List") para aplicarmos nos dados diários.

In [9]:
print("📂 Carregando cadastro de fundos...")

# Lê o cadastro (encoding 'latin1' é padrão de arquivos governamentais antigos no BR)
df_cad = pd.read_csv(f'{RAW_DIR}/cad_fi.csv', sep=';', encoding='latin1', low_memory=False)

# Normaliza colunas para evitar erros de maiúsculas/minúsculas
df_cad.columns = df_cad.columns.str.upper()

# FILTRO 1: Apenas classe 'Ações'
# Obs: Na CVM, a classe exata geralmente vem como 'Fundo de Ações'
filtro_acoes = df_cad['CLASSE'] == 'Fundo de Ações'

# FILTRO 2: Apenas fundos em funcionamento (opcional, mas remove lixo)
filtro_ativo = df_cad['SIT'] == 'EM FUNCIONAMENTO NORMAL'

# Aplica filtros
df_acoes = df_cad[filtro_acoes & filtro_ativo].copy()

# Cria a lista de CNPJs permitidos (nossa "White List")
cnpjs_acoes = set(df_acoes['CNPJ_FUNDO'].unique())

print(f"✅ Total de fundos de Ações encontrados: {len(cnpjs_acoes)}")
print(f"Exemplos de Classes no arquivo: {df_cad['CLASSE'].unique()[:5]}")

📂 Carregando cadastro de fundos...
✅ Total de fundos de Ações encontrados: 0
Exemplos de Classes no arquivo: [nan 'Multimercado' 'Ações' 'Renda Fixa' 'Referenciado']


## 3. Loop de Processamento (O "Triturador")

Agora iteramos sobre todos os arquivos mensais baixados. Para cada arquivo:

1 - Carregamos o CSV na memória.
2 - Filtramos mantendo apenas os CNPJs de Ações identificados no passo anterior.
3 - Acumulamos o resultado.

Otimização: Usamos gc.collect() para liberar memória RAM entre as iterações, evitando que o computador trave com o volume de dados.

In [10]:
arquivos_diarios = sorted(glob.glob(f'{RAW_DIR}/inf_diario_fi_*.csv'))
dfs_filtrados = []

print(f"🚀 Iniciando processamento de {len(arquivos_diarios)} arquivos mensais...")

colunas_importantes = ['CNPJ_FUNDO', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST']

for arquivo in arquivos_diarios:
    try:
        # Lê o arquivo
        df_mes = pd.read_csv(arquivo, sep=';', encoding='latin1', low_memory=False)

        # 1. Normaliza colunas (Maiúsculas e sem espaços)
        df_mes.columns = df_mes.columns.str.upper().str.strip()

        # 2. ADAPTAÇÃO RESOLUÇÃO 175: Renomeia CNPJ_FUNDO_CLASSE para CNPJ_FUNDO
        if 'CNPJ_FUNDO_CLASSE' in df_mes.columns and 'CNPJ_FUNDO' not in df_mes.columns:
            df_mes.rename(columns={'CNPJ_FUNDO_CLASSE': 'CNPJ_FUNDO'}, inplace=True)

        # 3. Verifica se a coluna CNPJ agora existe
        if 'CNPJ_FUNDO' in df_mes.columns:
            # Filtra pelos CNPJs de Ações (nossa "White List")
            df_mes = df_mes[df_mes['CNPJ_FUNDO'].isin(cnpjs_acoes)]

            if not df_mes.empty:
                # Seleciona apenas colunas essenciais para economizar memória e evitar erros de colunas extras
                cols_to_keep = [c for c in colunas_importantes if c in df_mes.columns]
                df_mes = df_mes[cols_to_keep]

                dfs_filtrados.append(df_mes)
        else:
            # Mostra as primeiras 5 colunas para entendermos o que veio
            print(f"⚠️ Pulei {os.path.basename(arquivo)}: Coluna CNPJ não encontrada. Headers: {list(df_mes.columns)[:5]}")

        # Limpeza de memória
        del df_mes
        gc.collect()

    except Exception as e:
        print(f"❌ Erro ao ler {os.path.basename(arquivo)}: {e}")

# Concatenação Final
if len(dfs_filtrados) > 0:
    print("🔄 Concatenando todos os meses...")
    df_final = pd.concat(dfs_filtrados, ignore_index=True)

    # Tratamento final de tipos
    df_final['DT_COMPTC'] = pd.to_datetime(df_final['DT_COMPTC'])
    df_final.sort_values(by=['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)

    # Salvar
    arquivo_saida = f'{PROCESSED_DIR}/base_acoes_consolidada.parquet'
    df_final.to_parquet(arquivo_saida, index=False)
    print(f"✅ SUCESSO! Base salva: {df_final.shape[0]} linhas. Arquivo: {arquivo_saida}")
else:
    print("💀 ERRO: Nenhum dado restou após os filtros.")

🚀 Iniciando processamento de 25 arquivos mensais...
💀 ERRO: Nenhum dado restou após os filtros.


## 4. Salvando em Parquet


Salvamos o resultado consolidado em formato Parquet.
Por que Parquet? Ocupa ~80% menos espaço em disco que CSV.
Mantém os tipos de dados (datas, números) corretos, evitando ter que converter tudo de novo na próxima etapa.

In [11]:
# Converte a coluna de data para datetime (essencial para séries temporais)
df_final['DT_COMPTC'] = pd.to_datetime(df_final['DT_COMPTC'])

# Ordena por Fundo e Data
df_final.sort_values(by=['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)

arquivo_saida = f'{PROCESSED_DIR}/base_acoes_consolidada.parquet'
print(f"💾 Salvando em: {arquivo_saida}")

# Salva em Parquet (requer biblioteca pyarrow ou fastparquet instalada)
df_final.to_parquet(arquivo_saida, index=False)

print("✅ Processo de Limpeza Concluído!")

NameError: name 'df_final' is not defined